<a href="https://colab.research.google.com/github/MichelyTamessawa/classificacao-laranja/blob/master/classificacao_laranjas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

# **Leitura do Dataset**

In [ ]:
def leitura(diretorio):
  siglas_laranjas = ["B", "C", "D", "P", "V"]
  classes_laranjas = ["Boa", "Casca Grossa", "Dano Praga", "Podre", "Verde"]
  caminho_drive = "/content/drive/MyDrive/4 ano/tcc/classificação de laranjas/"

  # inicialização dos objetos do tipo DataFrame que vão armazenar os dados da base de dados
  boa = pd.DataFrame()
  casca_grossa = pd.DataFrame()
  dano_praga = pd.DataFrame()
  podre = pd.DataFrame()
  verde = pd.DataFrame()

  for i, classe in enumerate(classes_laranjas):
    for j in range(1, 11):

      # caminho de diretórios até o arquivo da base  
      caminho = caminho_drive + diretorio + classe + "/Fold " + str(j) + siglas_laranjas[i] + ".csv"
      
      # leiturclassea de uma linha do arquivo 
      linha = pd.read_csv(caminho)
      
      # a linha que foi lida é adicionada ao DataFrame da sua classe
      if classe == "Boa":
          boa = pd.concat([boa, linha],ignore_index=True)
      elif classe == "Casca Grossa":
          casca_grossa = pd.concat([casca_grossa, linha],ignore_index=True)
      elif classe == "Dano Praga":
          dano_praga = pd.concat([dano_praga, linha],ignore_index=True)
      elif classe == "Podre":
          podre = pd.concat([podre, linha],ignore_index=True)
      elif classe == "Verde":
          verde = pd.concat([verde, linha],ignore_index=True)
  
  return boa, casca_grossa, dano_praga, podre, verde

# **Classificação Multiclasses**

In [ ]:
def todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  df = pd.concat([boa, casca_grossa, dano_praga, podre, verde], axis=0, ignore_index=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]
  
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test

# **Classificação Binária: (Um-vs-Todos)**

## Divisão de inteiros por 4

In [ ]:
def divide_quatro_partes(num):
  resto = num % 4

  if resto == 0:
      return [num//4, num//4, num//4, num//4]
  elif resto == 3:
      return [num//4+1, num//4+1, num//4+1, num//4]
  elif resto == 2:
      return [num//4+1, num//4+1, num//4, num//4]
  elif resto == 1:
      return [num//4+1, num//4, num//4, num//4]

## Divisão de inteiros por 3

In [ ]:
def divide_tres_partes(num):
    resto = num % 3

    if resto == 0:
        return [num//3, num//3, num//3]
    elif resto == 2:
        return [num//3+1, num//3+1, num//3]
    elif resto == 1:
        return [num//3+1, num//3, num//3]

## Divisão de inteiros por 2

In [ ]:
def divide_duas_partes(num):
    resto = num % 2

    if resto == 0:
        return [num//2, num//2]
    elif resto == 1:
        return [num//2+1, num//2]

## Balanceamento das classes

In [ ]:
def boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = boa.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  c = casca_grossa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)

  df = pd.concat([boa, c, d, p, v], axis=0, ignore_index=True)
  df["class"].replace({2:1, 3:1, 4:1}, inplace=True)

  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = casca_grossa.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)
  
  df = pd.concat([casca_grossa, b, d, p, v], axis=0, ignore_index=True)
  df["class"].replace({0:1, 1:0, 2:1, 3:1, 4:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = dano_praga.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)
    
  df = pd.concat([dano_praga, c, b, p, v], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:0, 3:1, 4:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = podre.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  d = dano_praga.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)
    
  df = pd.concat([podre, c, b, d, v], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:1, 3:0, 4:1}, inplace=True)

  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = verde.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  d = dano_praga.sample(n=divisao_classes[2],random_state=1)
  p = podre.sample(n=divisao_classes[3],random_state=1)

  df = pd.concat([verde, c, b, p, d], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:1, 3:1, 4:0}, inplace=True)  

  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test

## Balanceamento das classes removendo a Verde (RV)

In [ ]:
def boa_vs_todosRV(boa, casca_grossa, dano_praga, podre):
  n_classe  = boa.shape[0]
  divisao_classes = divide_tres_partes(n_classe)

  c = casca_grossa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)

  df = pd.concat([boa, c, d, p], axis=0, ignore_index=True)
  df["class"].replace({2:1, 3:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre):
  n_classe  = casca_grossa.shape[0]
  divisao_classes = divide_tres_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)

  df = pd.concat([casca_grossa, b, d, p], axis=0, ignore_index=True)
  df["class"].replace({0:1, 1:0, 2:1, 3:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre):
  n_classe  = dano_praga.shape[0]
  divisao_classes = divide_tres_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
    
  df = pd.concat([dano_praga, c, b, p], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:0, 3:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def podre_vs_todosRV(boa, casca_grossa, dano_praga, podre):
  n_classe  = podre.shape[0]
  divisao_classes = divide_tres_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  d = dano_praga.sample(n=divisao_classes[2],random_state=1)
      
  df = pd.concat([podre, c, b, d], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:1, 3:0}, inplace=True)

  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test

## Balanceamento das classes removendo a Casca Grossa (RCG)

In [ ]:
def boa_vs_todosRCG(boa, dano_praga, podre):
  n_classe  = boa.shape[0]
  divisao_classes = divide_duas_partes(n_classe)

  d = dano_praga.sample(n=divisao_classes[0],random_state=1)
  p = podre.sample(n=divisao_classes[1],random_state=1)

  df = pd.concat([boa, d, p], axis=0, ignore_index=True)
  df["class"].replace({2:1, 3:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def dano_praga_vs_todosRCG(boa, dano_praga, podre):
  n_classe  = dano_praga.shape[0]
  divisao_classes = divide_duas_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  p = podre.sample(n=divisao_classes[1],random_state=1)
    
  df = pd.concat([dano_praga, b, p], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:0, 3:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def podre_vs_todosRCG(boa, dano_praga, podre):
  n_classe  = podre.shape[0]
  divisao_classes = divide_duas_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
      
  df = pd.concat([podre, b, d], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:1, 3:0}, inplace=True)

  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test

# **Classificadores**

## K-NN

In [ ]:
def knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names):
  # Normalizacao dos dados
  scaler = MinMaxScaler()
  X_train_scaled = scaler.fit_transform(x_train)
  X_test_scaled = scaler.transform(x_test)
  
  knn = KNeighborsClassifier()
  
  param_grid = {
      'n_neighbors': range(1,30),
      'leaf_size': (20,40,1),
      'p': (1,2),
      'weights': ('uniform', 'distance'),
      'metric': ('minkowski', 'chebyshev') }

  # Treinamento com gridsearch
  grid = GridSearchCV(estimator=knn, param_grid=param_grid, cv=10, refit='True', scoring='accuracy')
  grid_search = grid.fit(X_train_scaled, y_train)
  print("Parametros: {}".format(grid_search.best_params_))
  
  # Teste
  y_test_result = grid.predict(X_test_scaled)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acuracia dos testes: {:.2f}%".format(test_accuracy))

  # Matriz de Confusao
  print("Matriz de Confusao:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))

  # Relatorio de Classificacao 
  print("Relatorio de Classificacao:")
  print(classification_report(y_test, y_test_result, target_names=target_names))

## Random Forest

In [ ]:
def rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names):
  # Normalizacao dos dados
  scaler = MinMaxScaler()
  X_train_scaled = scaler.fit_transform(x_train)
  X_test_scaled = scaler.transform(x_test)

  rf = RandomForestClassifier(random_state=0)
  
  param_grid = {
    'max_depth': [10, 50],
    'min_samples_leaf': [1,3],
    'min_samples_split': [2,5],
    'n_estimators': [100, 200]
  }

  # Treinamento com gridsearch
  grid = GridSearchCV(rf, param_grid, cv=10, refit='True', scoring='accuracy')
  grid_search = grid.fit(X_train_scaled, y_train)
  print("Parametros: {}".format(grid_search.best_params_))

  # Teste
  y_test_result = grid.predict(X_test_scaled)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acuracia dos testes: {:.2f}%".format(test_accuracy))

  # Matriz de Confusao
  print("Matriz de Confusao:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))

  # Relatorio de Classificacao 
  print("Relatorio de Classificacao:")
  print(classification_report(y_test, y_test_result, target_names=target_names))


## Classificador SVM

In [ ]:
def svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names):
  # Normalizacao dos dados
  scaler = MinMaxScaler()
  X_train_scaled = scaler.fit_transform(x_train)
  X_test_scaled = scaler.transform(x_test)

  svm = SVC()
  
  param_grid = [
    {"kernel": ['rbf'], "gamma": [1, 0.1, 0.01, 0.001, 0.0001], "C": [0.1, 1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
  ]
              
  # Treinamento com gridsearch
  grid = GridSearchCV(svm, param_grid, cv=10, refit=True, scoring='accuracy', n_jobs=-1)
  grid_search = grid.fit(X_train_scaled, y_train)
  print("Parametros: {}".format(grid_search.best_params_))

  # Teste
  y_test_result = grid.predict(X_test_scaled)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acuracia dos testes: {:.2f}%".format(test_accuracy))

  # Matriz de Confusao
  print("Matriz de Confusao:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))

  # Relatorio de Classificacao 
  print("Relatorio de Classificacao:")
  print(classification_report(y_test, y_test_result, target_names=target_names))

# **>> Execuções <<**

# **Multiclasses**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Acurácia dos testes: 71.11%
Matriz de Confusão:
Predito    0    1    2    3    4   All
Real                                  
0        328   54   31   35   24   472
1         30  344   38   40    9   461
2         31   38  309   29   23   430
3         28   65   40  285   26   444
4         25   12   47   28  341   453
All      442  513  465  417  423  2260
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.74      0.69      0.72       472
           1       0.67      0.75      0.71       461
           2       0.66      0.72      0.69       430
           3       0.68      0.64      0.66       444
           4       0.81      0.75      0.78       453

    accuracy                           0.71      2260
   macro avg       0.71      0.71      0.71      2260
weighted avg       0.71      0.71      0.71      2260



### RGB

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
Acurácia dos testes: 88.01%
Matriz de Confusão:
Predito    0    1    2    3    4   All
Real                                  
0        398   22   27   17    8   472
1         11  418   11   15    6   461
2         16    9  379   26    0   430
3         33   25   36  350    0   444
4          2    5    1    1  444   453
All      460  479  454  409  458  2260
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.87      0.84      0.85       472
           1       0.87      0.91      0.89       461
           2       0.83      0.88      0.86       430
           3       0.86      0.79      0.82       444
           4       0.97      0.98      0.97       453

    accuracy                           0.88      2260
   macro avg       0.88      0.88      0.88      2260
weighted avg       0.88      0.88      0.88      2260



## RandomForest

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 76.02%
Matriz de Confusão:
Predito    0    1    2    3    4   All
Real                                  
0        368   47   29   16   12   472
1         29  357   34   39    2   461
2         21   34  335   22   18   430
3         32   49   46  298   19   444
4         19   14   36   24  360   453
All      469  501  480  399  411  2260
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.78      0.78      0.78       472
           1       0.71      0.77      0.74       461
           2       0.70      0.78      0.74       430
           3       0.75      0.67      0.71       444
           4       0.88      0.79      0.83       453

    accuracy                           0.76      2260
   macro avg       0.76      0.76      0.76      2260
weighted avg       0.76      0.76      0.76      2260



### RGB

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 88.54%
Matriz de Confusão:
Predito    0    1    2    3    4   All
Real                                  
0        387   24   35   22    4   472
1         14  417   14   15    1   461
2         12   12  386   20    0   430
3         19   17   40  368    0   444
4          4    5    1    0  443   453
All      436  475  476  425  448  2260
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.89      0.82      0.85       472
           1       0.88      0.90      0.89       461
           2       0.81      0.90      0.85       430
           3       0.87      0.83      0.85       444
           4       0.99      0.98      0.98       453

    accuracy                           0.89      2260
   macro avg       0.89      0.89      0.89      2260
weighted avg       0.89      0.89      0.89      2260



## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 78.45%
Matriz de Confusão:
Predito    0    1    2    3    4   All
Real                                  
0        349   47   33   28   15   472
1         33  369   27   30    2   461
2         22   21  367   17    3   430
3         44   42   42  299   17   444
4         23    6    5   30  389   453
All      471  485  474  404  426  2260
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.74      0.74      0.74       472
           1       0.76      0.80      0.78       461
           2       0.77      0.85      0.81       430
           3       0.74      0.67      0.71       444
           4       0.91      0.86      0.89       453

    accuracy                           0.78      2260
   macro avg       0.79      0.79      0.78      2260
weighted avg       0.79      0.78      0.78      2260



### RGB

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 95.00%
Matriz de Confusão:
Predito    0    1    2    3    4   All
Real                                  
0        438    7   15   10    2   472
1          8  444    3    4    2   461
2         12    0  411    7    0   430
3         21    7   11  405    0   444
4          1    3    0    0  449   453
All      480  461  440  426  453  2260
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       472
           1       0.96      0.96      0.96       461
           2       0.93      0.96      0.94       430
           3       0.95      0.91      0.93       444
           4       0.99      0.99      0.99       453

    accuracy                           0.95      2260
   macro avg       0.95      0.95      0.95      2260
weighted avg       0.95      0.95      0.95      2260



# **Multiclasses: Removendo a Verde**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Acurácia dos testes: 73.98%
Matriz de Confusão:
Predito    0    1    2    3   All
Real                             
0        356   47   43   24   470
1         35  330   33   44   442
2         43   42  349   31   465
3         43   46   39  301   429
All      477  465  464  400  1806
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.75      0.76      0.75       470
           1       0.71      0.75      0.73       442
           2       0.75      0.75      0.75       465
           3       0.75      0.70      0.73       429

    accuracy                           0.74      1806
   macro avg       0.74      0.74      0.74      1806
weighted avg       0.74      0.74      0.74      1806



### RGB

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 4, 'p': 2, 'weights': 'distance'}
Acurácia dos testes: 86.21%
Matriz de Confusão:
Predito    0    1    2    3   All
Real                             
0        396   24   31   19   470
1         17  400   14   11   442
2         14   16  413   22   465
3         28   28   25  348   429
All      455  468  483  400  1806
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       470
           1       0.85      0.90      0.88       442
           2       0.86      0.89      0.87       465
           3       0.87      0.81      0.84       429

    accuracy                           0.86      1806
   macro avg       0.86      0.86      0.86      1806
weighted avg       0.86      0.86      0.86      1806



## RandomForest

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 75.80%
Matriz de Confusão:
Predito    0    1    2    3   All
Real                             
0        352   52   36   30   470
1         33  344   39   26   442
2         21   44  376   24   465
3         35   56   41  297   429
All      441  496  492  377  1806
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.80      0.75      0.77       470
           1       0.69      0.78      0.73       442
           2       0.76      0.81      0.79       465
           3       0.79      0.69      0.74       429

    accuracy                           0.76      1806
   macro avg       0.76      0.76      0.76      1806
weighted avg       0.76      0.76      0.76      1806



### RGB

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 85.33%
Matriz de Confusão:
Predito    0    1    2    3   All
Real                             
0        385   29   34   22   470
1         17  404   15    6   442
2         14   11  412   28   465
3         25   23   41  340   429
All      441  467  502  396  1806
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.87      0.82      0.85       470
           1       0.87      0.91      0.89       442
           2       0.82      0.89      0.85       465
           3       0.86      0.79      0.82       429

    accuracy                           0.85      1806
   macro avg       0.85      0.85      0.85      1806
weighted avg       0.85      0.85      0.85      1806



## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 76.69%
Matriz de Confusão:
Predito    0    1    2    3   All
Real                             
0        361   44   34   31   470
1         36  345   22   39   442
2         20   35  381   29   465
3         44   45   42  298   429
All      461  469  479  397  1806
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.78      0.77      0.78       470
           1       0.74      0.78      0.76       442
           2       0.80      0.82      0.81       465
           3       0.75      0.69      0.72       429

    accuracy                           0.77      1806
   macro avg       0.77      0.77      0.77      1806
weighted avg       0.77      0.77      0.77      1806



### RGB

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 93.19%
Matriz de Confusão:
Predito    0    1    2    3   All
Real                             
0        433    6   23    8   470
1          7  423    3    9   442
2         14    6  436    9   465
3         20   11    7  391   429
All      474  446  469  417  1806
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.91      0.92      0.92       470
           1       0.95      0.96      0.95       442
           2       0.93      0.94      0.93       465
           3       0.94      0.91      0.92       429

    accuracy                           0.93      1806
   macro avg       0.93      0.93      0.93      1806
weighted avg       0.93      0.93      0.93      1806



# **Multiclasses: Removendo a Casca Grossa**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa[0:0], dano_praga, podre,  verde[0:0])
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 4, 'p': 2, 'weights': 'distance'}
Acurácia dos testes: 78.03%
Matriz de Confusão:
Predito    0    2    3   All
Real                        
0        376   45   40   461
2         49  350   44   443
3         58   61  329   448
All      483  456  413  1352
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       461
           2       0.77      0.79      0.78       443
           3       0.80      0.73      0.76       448

    accuracy                           0.78      1352
   macro avg       0.78      0.78      0.78      1352
weighted avg       0.78      0.78      0.78      1352



### RGB

In [ ]:
target_names = ['0','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa[0:0], dano_praga, podre,  verde[0:0])
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
Acurácia dos testes: 85.95%
Matriz de Confusão:
Predito    0    2    3   All
Real                        
0        403   31   27   461
2         26  385   32   443
3         42   32  374   448
All      471  448  433  1352
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.86      0.87      0.86       461
           2       0.86      0.87      0.86       443
           3       0.86      0.83      0.85       448

    accuracy                           0.86      1352
   macro avg       0.86      0.86      0.86      1352
weighted avg       0.86      0.86      0.86      1352



## RandomForest

### Tons de Cinza

In [ ]:
target_names = ['0','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa[0:0], dano_praga, podre,  verde[0:0])
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Acurácia dos testes: 81.14%
Matriz de Confusão:
Predito    0    2    3   All
Real                        
0        384   45   32   461
2         32  371   40   443
3         51   55  342   448
All      467  471  414  1352
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       461
           2       0.79      0.84      0.81       443
           3       0.83      0.76      0.79       448

    accuracy                           0.81      1352
   macro avg       0.81      0.81      0.81      1352
weighted avg       0.81      0.81      0.81      1352



### RGB

In [ ]:
target_names = ['0','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa[0:0], dano_praga, podre,  verde[0:0])
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 87.06%
Matriz de Confusão:
Predito    0    2    3   All
Real                        
0        394   43   24   461
2         14  401   28   443
3         26   40  382   448
All      434  484  434  1352
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.91      0.85      0.88       461
           2       0.83      0.91      0.87       443
           3       0.88      0.85      0.87       448

    accuracy                           0.87      1352
   macro avg       0.87      0.87      0.87      1352
weighted avg       0.87      0.87      0.87      1352



## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa[0:0], dano_praga, podre,  verde[0:0])
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 84.47%
Matriz de Confusão:
Predito    0    2    3   All
Real                        
0        391   32   38   461
2         27  393   23   443
3         45   45  358   448
All      463  470  419  1352
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.84      0.85      0.85       461
           2       0.84      0.89      0.86       443
           3       0.85      0.80      0.83       448

    accuracy                           0.84      1352
   macro avg       0.85      0.84      0.84      1352
weighted avg       0.85      0.84      0.84      1352



### RGB

In [ ]:
target_names = ['0','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa[0:0], dano_praga, podre,  verde[0:0])
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 93.42%
Matriz de Confusão:
Predito    0    2    3   All
Real                        
0        427   24   10   461
2         18  416    9   443
3         16   12  420   448
All      461  452  439  1352
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       461
           2       0.92      0.94      0.93       443
           3       0.96      0.94      0.95       448

    accuracy                           0.93      1352
   macro avg       0.93      0.93      0.93      1352
weighted avg       0.93      0.93      0.93      1352



# **Binária**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 6, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 81.41%
Acurácia dos testes: 79.74%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        402   69  471
1        115  322  437
All      517  391  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.78      0.85      0.81       471
           1       0.82      0.74      0.78       437

    accuracy                           0.80       908
   macro avg       0.80      0.80      0.80       908
weighted avg       0.80      0.80      0.80       908



Casca Grossa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 82.46%
Acurácia dos testes: 84.25%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        416   55  471
1         88  349  437
All      

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
Acurácia para os dados de treinamento: 87.80%
Acurácia dos testes: 87.67%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        427   44  471
1         68  369  437
All      495  413  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       471
           1       0.89      0.84      0.87       437

    accuracy                           0.88       908
   macro avg       0.88      0.88      0.88       908
weighted avg       0.88      0.88      0.88       908



Casca Grossa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 93.03%
Acurácia dos testes: 93.06%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        458   13  471
1         50  387  437
All      5

## RandomForest

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia no treinamento: 83.42%
Acurácia dos testes: 81.83%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        387   84  471
1         81  356  437
All      468  440  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.83      0.82      0.82       471
           1       0.81      0.81      0.81       437

    accuracy                           0.82       908
   macro avg       0.82      0.82      0.82       908
weighted avg       0.82      0.82      0.82       908



Casca Grossa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Acurácia no treinamento: 83.89%
Acurácia dos testes: 84.69%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        414   57  471
1         82  355  437
All      496  412  908
Relatorio de Classific

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia no treinamento: 88.85%
Acurácia dos testes: 87.89%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        410   61  471
1         49  388  437
All      459  449  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       471
           1       0.86      0.89      0.88       437

    accuracy                           0.88       908
   macro avg       0.88      0.88      0.88       908
weighted avg       0.88      0.88      0.88       908



Casca Grossa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia no treinamento: 93.09%
Acurácia dos testes: 94.27%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        457   14  471
1         38  399  437
All      495  413  908
Relatorio de Classific

## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia no treinamento: 86.04%
Acurácia dos testes: 84.03%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        412   59  471
1         86  351  437
All      498  410  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       471
           1       0.86      0.80      0.83       437

    accuracy                           0.84       908
   macro avg       0.84      0.84      0.84       908
weighted avg       0.84      0.84      0.84       908



Casca Grossa vs Todos
Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia no treinamento: 87.22%
Acurácia dos testes: 88.33%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        434   37  471
1         69  368  437
All      503  405  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.86      

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia no treinamento: 92.10%
Acurácia dos testes: 92.29%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        437   34  471
1         36  401  437
All      473  435  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.92      0.93      0.93       471
           1       0.92      0.92      0.92       437

    accuracy                           0.92       908
   macro avg       0.92      0.92      0.92       908
weighted avg       0.92      0.92      0.92       908



Casca Grossa vs Todos
Parâmetros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Acurácia no treinamento: 96.17%
Acurácia dos testes: 96.59%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        462    9  471
1         22  415  437
All      484  424  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.95      0

# **Binária: Removendo a Verde**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 8, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 81.66%
Acurácia dos testes: 81.06%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        393   78  471
1         94  343  437
All      487  421  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       471
           1       0.81      0.78      0.80       437

    accuracy                           0.81       908
   macro avg       0.81      0.81      0.81       908
weighted avg       0.81      0.81      0.81       908



Casca Grossa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 81.14%
Acurácia dos testes: 83.59%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        400   71  471
1         78  359  437
All      

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 87.17%
Acurácia dos testes: 86.56%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        419   52  471
1         70  367  437
All      489  419  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       471
           1       0.88      0.84      0.86       437

    accuracy                           0.87       908
   macro avg       0.87      0.86      0.87       908
weighted avg       0.87      0.87      0.87       908



Casca Grossa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 91.77%
Acurácia dos testes: 93.50%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        450   21  471
1         38  399  437
All      

## RandomForest 

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia no treinamento: 84.58%
Acurácia dos testes: 82.82%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        382   89  471
1         67  370  437
All      449  459  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.85      0.81      0.83       471
           1       0.81      0.85      0.83       437

    accuracy                           0.83       908
   macro avg       0.83      0.83      0.83       908
weighted avg       0.83      0.83      0.83       908



Casca Grossa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Acurácia no treinamento: 81.99%
Acurácia dos testes: 83.92%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        400   71  471
1         75  362  437
All      475  433  908
Relatorio de Classific

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Acurácia no treinamento: 87.94%
Acurácia dos testes: 86.89%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        401   70  471
1         49  388  437
All      450  458  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.89      0.85      0.87       471
           1       0.85      0.89      0.87       437

    accuracy                           0.87       908
   macro avg       0.87      0.87      0.87       908
weighted avg       0.87      0.87      0.87       908



Casca Grossa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Acurácia no treinamento: 92.59%
Acurácia dos testes: 94.71%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        448   23  471
1         25  412  437
All      473  435  908
Relatorio de Classific

## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia no treinamento: 85.00%
Acurácia dos testes: 85.46%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        404   67  471
1         65  372  437
All      469  439  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.86      0.86      0.86       471
           1       0.85      0.85      0.85       437

    accuracy                           0.85       908
   macro avg       0.85      0.85      0.85       908
weighted avg       0.85      0.85      0.85       908



Casca Grossa vs Todos
Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia no treinamento: 85.68%
Acurácia dos testes: 89.21%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        426   45  471
1         53  384  437
All      479  429  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.89      

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Acurácia no treinamento: 91.44%
Acurácia dos testes: 91.41%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        427   44  471
1         34  403  437
All      461  447  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       471
           1       0.90      0.92      0.91       437

    accuracy                           0.91       908
   macro avg       0.91      0.91      0.91       908
weighted avg       0.91      0.91      0.91       908



Casca Grossa vs Todos
Parâmetros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Acurácia no treinamento: 96.26%
Acurácia dos testes: 97.69%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        463    8  471
1         13  424  437
All      476  432  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.97      0.

# **Binária: Removendo a Casca Grossa**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRCG(boa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRCG(boa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRCG(boa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
Acurácia dos testes: 81.83%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        392   79  471
1         86  351  437
All      478  430  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       471
           1       0.82      0.80      0.81       437

    accuracy                           0.82       908
   macro avg       0.82      0.82      0.82       908
weighted avg       0.82      0.82      0.82       908



Dano Praga vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 4, 'p': 2, 'weights': 'distance'}
Acurácia dos testes: 83.67%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        398   66  464
1         79  345  424
All      477  411  888
Relatorio de Classificacao:
              precision    recall  f1-score   suppor

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRCG(boa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRCG(boa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRCG(boa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Acurácia dos testes: 88.99%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        424   47  471
1         53  384  437
All      477  431  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       471
           1       0.89      0.88      0.88       437

    accuracy                           0.89       908
   macro avg       0.89      0.89      0.89       908
weighted avg       0.89      0.89      0.89       908



Dano Praga vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Acurácia dos testes: 89.30%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        419   45  464
1         50  374  424
All      469  419  888
Relatorio de Classificacao:
              precision    recall  f1-score   support


## RandomForest 

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRCG(boa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRCG(boa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRCG(boa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 84.69%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        393   78  471
1         61  376  437
All      454  454  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.87      0.83      0.85       471
           1       0.83      0.86      0.84       437

    accuracy                           0.85       908
   macro avg       0.85      0.85      0.85       908
weighted avg       0.85      0.85      0.85       908



Dano Praga vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 85.25%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        396   68  464
1         63  361  424
All      459  429  888
Relatorio de Classificacao:
              precision    recall  f1-score   support

     

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRCG(boa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRCG(boa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRCG(boa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 87.44%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        410   61  471
1         53  384  437
All      463  445  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       471
           1       0.86      0.88      0.87       437

    accuracy                           0.87       908
   macro avg       0.87      0.87      0.87       908
weighted avg       0.87      0.87      0.87       908



Dano Praga vs Todos
Parâmetros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acurácia dos testes: 88.29%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        414   50  464
1         54  370  424
All      468  420  888
Relatorio de Classificacao:
              precision    recall  f1-score   support

     

## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRCG(boa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRCG(boa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRCG(boa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 86.12%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        416   55  471
1         71  366  437
All      487  421  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.85      0.88      0.87       471
           1       0.87      0.84      0.85       437

    accuracy                           0.86       908
   macro avg       0.86      0.86      0.86       908
weighted avg       0.86      0.86      0.86       908



Dano Praga vs Todos
Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 88.96%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        417   47  464
1         51  373  424
All      468  420  888
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       464
           1       0.89      0.88      0.

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRCG(boa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRCG(boa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRCG(boa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parâmetros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 92.62%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        435   36  471
1         31  406  437
All      466  442  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.93      0.92      0.93       471
           1       0.92      0.93      0.92       437

    accuracy                           0.93       908
   macro avg       0.93      0.93      0.93       908
weighted avg       0.93      0.93      0.93       908



Dano Praga vs Todos
Parâmetros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acurácia dos testes: 94.93%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        442   22  464
1         23  401  424
All      465  423  888
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       464
           1       0.95      0.95      0.9